<h1>Correlação Entre Variáveis no Roteamento e atribuição de espectro</h1>
<p style='font-size: 20px' >Alunos: André Luiz Maso, Artur Bernardo Mallmann, Maikon Douglas Pereira e Tatiane Barbosa</p>
    
<p>&emsp;&emsp;Neste Trabalho vamos explorar qual é a correlação entre Taxa de Bloqueio (Tb = B/D), onde B é o número de bloqueios ocorridos durante o RSA (Roteamento e atribuição de espectro) e D é o número total de demandas (soma da triangular superior da matriz de demandas), e o Índice de Wiener da topologia (Soma do comprimento dos enlaces dos caminhos mais curtos)</p>

In [1]:
import numpy as np
import math
from random import *
#pandas para base de dados
import pandas

<h1>Calculo de Coorelação:</h1>

In [2]:
def corr(x,y,p=False):
    # n
    n = len(x)
#Somatórios:
    #superiores
    xysum = np.dot(x,y) #soma dos x*y
    xsum = sum(x)
    ysum = sum(y)
    #inferiores
    #soma dos quadrados
    x2sum = np.dot(x,x) #multiplica entrada a entrada dos vetores x e ele mesmo e soma
    y2sum = sum([v*v for v in y]) #forma na unha de fazer o mesmo
    #quadrado das somas:
    xsum2 = xsum**2
    ysum2 = ysum**2
#formula
    #parte superior:
    sup = (xysum - xsum*ysum/n)
    #parte inferior:
    infe = math.sqrt ( (x2sum - xsum2/n) * (y2sum - ysum2/n) )
    corresult = sup/infe
    if(p):
        print("Taxa de correlação entre [x] e [y] é:")
        print("{}\n".format(sup)+len(str(sup))*"-"+" =  {}".format(corresult)+"\n{}".format(infe))
    return corresult

<h2>Teste Da Correlação</h2>
<p>Para calcular a correlação entre os valores de duas variáveis basta entrar com dois vetores com os valores a serem comparados:
corr(vec1,vec2)</p>

In [3]:
gerax = lambda base,n,dif: [base + x*dif for x in range(n)] # modo preguiçoso de gerar n números espaçados em 0.5
#Carregar valores(teste):
x = gerax(2.5, 6,0.5)
y = [0.4,0.4,0.3,0.2,0.2,0.1]
corr(x,y,True)

Taxa de correlação entre [x] e [y] é:
-0.5499999999999998
------------------- =  -0.9710083124552239
0.5664215155988811


-0.9710083124552239

<h1>Carregamento de Dados</h1>
<p>Nesta sessão do código é feita a leitura dos arquivos .csv</p>

In [4]:
#!/usr/bin/python

from pathlib import Path

directory = './nodes_links/'
allnodes = dict()
alllinks = dict()
names = list()
for path in Path(directory).iterdir():
    url = str(path)
    netname = path.name.split('_')[0]
    if(path.name.count('nodes')==1):
        p = allnodes
    else:
        p = alllinks
    try:
        item = pandas.read_csv(url)
        p.update( {netname : item })
    #    print(url+" lido")
    except:
        try:
            #mudar o encoding para os caracteres
            item = pandas.read_csv(url,encoding="iso-8859-1")
            p.update( {netname : item })
        #    print(url+" lido")
        except:
            print("!!! não foi possivel ler > "+name+" < !!!")
            pass
    if names.count(netname) == 0:
        names.append(netname) 


<p>Todos os links e nós ficam guardados nos dicts alllinks e allnodes, e para serem lidos basta passar o prefixo dos arquivos originais entre colchetes. Estes prefixos são os nomes das nossas redes e ficam salvos no vetor names.</p>

In [5]:
print(names)

['canarie', 'italy', 'vbns', 'loni', 'germany', 'geant2', 'arpanet', 'cesnet', 'newnet', 'rnpBrazil', 'pionier', 'spain', 'renater', 'lambdaRailUsa', 'metrona', 'arnes', 'bren', 'portugal', 'usaGde', 'sanet', 'nsfnet', 'austria', 'viaDatacenterNet', 'coxUsa', 'mzima', 'scteste', 'eon', 'memorexEurope', 'deutschTelecom', 'internet2Usa', 'OmnicomEurope']


<p>Abaixo temos o exemplo do carregamento da rede cesnet:</p>

In [6]:
netlinks = alllinks['cesnet']
netnodes = allnodes['cesnet']

<p>Nos arquivos de nós temos no campo Id todos os nomes dos nós da rede, e nos links a origem e destino dos enlaces, como podemos ver abaixo:</p>

In [7]:
#u = pandas.read_csv("./nodes_links/renater_links.csv")
a = netnodes['Id'].values.tolist()
print("nodes:\n{}".format(a))
print("links:\n{}".format(netlinks))

nodes:
['Praga', 'Usti', 'Liberec', 'Hradec', 'Olomouc', 'Ostrava', 'Zlin', 'Brno', 'Jihlava', 'Budweiss', 'Pilsen', 'Pardubice']
links:
         From         To      Length  Capacity  Cost
0       Praga       Usti   44.320945        50     1
1        Usti    Liberec   57.122562        50     1
2     Liberec      Praga   57.111162        50     1
3       Praga     Pilsen  114.930257        50     1
4      Pilsen   Budweiss  123.277291        50     1
5    Budweiss      Praga  169.402099        50     1
6       Praga  Pardubice   72.350748        50     1
7   Pardubice     Hradec   69.868247        50     1
8      Hradec      Praga   91.221450        50     1
9      Hradec    Liberec   64.755029        50     1
10   Budweiss    Jihlava  106.688652        50     1
11    Jihlava       Brno   74.878684        50     1
12       Brno   Budweiss  163.463112        50     1
13    Olomouc     Hradec  136.245096        50     1
14    Olomouc       Zlin   30.591459        50     1
15       Zlin  

<h2>Gerando as matrizes de Adjacência</h2>
<p>Para montar as matrizes de adjacência consideramos a identificação numérica dos nós conforme a posição de cada um no vetor dos nós. Depois, com a ajuda da comparação de atributos da biblioteca pandas verificamos todas as origens e quais seus destinos, e assim preenchemos a matriz de adjacência.</p>
<p>A matriz de adjacência é bidirecional, para facilitar o uso com a implementação com qualquer algorítmo de busca a construção é nas diagonáis superiores e inferiores.</p>

In [8]:
def monta_matriz(netnodes,netlinks):
    nodes = netnodes['Id'].values.tolist()
    matrix = np.matrix(np.zeros( (len(nodes),len(nodes)),dtype=np.int ) )
    for n in nodes:
        l = netlinks.loc[netlinks['From']==n,['To']].values
        indexes = [nodes.index(x) for x in l]
        if(len(indexes) > 0):
            matrix[nodes.index(n),indexes] = 1
            matrix[indexes,nodes.index(n)] = 1 #bidirecional
    return matrix

matrix = monta_matriz(netnodes,netlinks)

print (matrix)

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


<p>Abaixo temos a construção das listas que representam os canais de comunicação dos enlaces. Estes canais são representados por 0's e 1's, salvamos em listas e depois guardamos em dicts que são acessados pela tupla que representa o origem e destino.</p>

In [9]:
def criar_canais(mtx,ncanais):
    canais = dict()
#     mtxc = np.matrix(np.zeros(mtx.shape), dtype=list)
    for i in range(len(mtx)):
        for j in range(len(mtx)):
            if(i<j):
                if(mtx[i,j] == 1):
                    lista = [1]*ncanais #1 livre, 0 ocupado
                    canais.update({(i,j):lista})
                    canais.update({(j,i):lista})#mesma instancia para as duas direções
    return canais

<p>Como os canais são bidirecionais os mesmos canais são usados nas duas direções, ou seja, do nó 0 para o nó 9 e do nó 9 para o nó 0 a mesma instância do objeto lista é utilizado. Assim se o canal 0 é ocupado em uma direção ele também não estará disponível no inverso do mesmo enlace, como podemos ver abaixo:</p> 

In [10]:
canais_rede = criar_canais(matrix,len(netnodes))
canais_rede[(9,0)][0]=0 # ocupar canal 0 do enlace
print (f'Ocupado\n{canais_rede[0,9]}\n{canais_rede[9,0]}')
canais_rede[(9,0)][0]=1 # desocupar canal 0 do enlace
print (f'Desocupado\n{canais_rede[0,9]}\n{canais_rede[9,0]}')

Ocupado
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Desocupado
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p>Excluir a diagonal inferior(caso necessário):</p>

In [11]:
def diagonalizar(mtx):
    nova = np.matrix(np.zeros(mtx.shape), dtype=int)
    for i in range(len(mtx)):
       # print(l)
        for j in range(len(mtx)):
            if(i<j):
                nova[i,j] = mtx[i,j]
    return nova
# print(diagonalizar(matrix))

<h2>Calculo e Incrementação do grau</h2>
<p>Para calcular o grau médio basta calcular a quantidade de elementos não nulos na linha ou coluna de cada nó, e depois dividir pelo núme de nós.</p>
<p><i>Obs. Isso se aplica quando a matriz é bidirecional e as diagonais superior e inferior são não nulas.</i></p>

In [12]:
def calc_grau(matrix):
    soma=0
    for l in range(len(matrix)):
        for c in range(len(matrix)):
            if matrix[l,c] != 0:
                soma += 1
    return soma/(len(matrix))

In [13]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 3.1666666666666665


<p>Para aumentar o grau da nossa topologia basta inserir novos enlaces na matriz de adjacência de forma aleatória. Para isso adiciona-se enlaces até o grafo atingir o grau desejado</p>

In [14]:
def aumentar_grau(mtx,inc_grau): #nodos = 6, p/incrementar 1 grau adicionamos 6 enlaces, para inc 0.5 add 3
    grau = calc_grau(mtx)
    mind = len(mtx)-1
    n_grau = grau
    while n_grau < mind and n_grau < grau + inc_grau:
        x,y=(None,None)
        while (x==None and y==None):
            x,y = (randint(0,mind), randint(0,mind))
            if x==y or mtx[x,y]==1:
                x,y=(None,None)
        mtx[x,y]=1
        mtx[y,x]=1 #bijetora...
        n_grau=calc_grau(mtx)

<p><b>Antes:</b></p>

In [15]:
print(matrix)

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


In [16]:
aumentar_grau(matrix,5)

<p><b>Depois:</b></p>

In [17]:
print(matrix)

[[0 1 1 1 1 0 1 1 0 1 1 1]
 [1 0 1 1 1 1 1 0 1 0 0 1]
 [1 1 0 1 0 0 1 1 1 0 0 0]
 [1 1 1 0 1 1 1 1 0 1 0 1]
 [1 1 0 1 0 1 1 1 1 1 1 0]
 [0 1 0 1 1 0 1 1 0 1 1 1]
 [1 1 1 1 1 1 0 1 1 0 1 1]
 [1 0 1 1 1 1 1 0 1 1 1 0]
 [0 1 1 0 1 0 1 1 0 1 1 1]
 [1 0 0 1 1 1 0 1 1 0 1 1]
 [1 0 0 0 1 1 1 1 1 1 0 0]
 [1 1 0 1 0 1 1 0 1 1 0 0]]


In [18]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 8.166666666666666


<h2>Construir todas as matrizes</h2>
<p>Nesta etapa geramos todas as matrizes à partir das informações tiradas dos arquivos nodes e links.

In [19]:
matrizes = dict()
#montar matrizes, canais, etc...
for x in names:
    netlinks = alllinks[x]
    netnodes = allnodes[x]
    matriz = monta_matriz(netnodes,netlinks)
    matrizes.update({x:matriz})
#     print("matriz {}:\n{}".format(x,matriz))

<p>Para acessar as matrizes basta passar o nome da rede entre []:</p>

In [20]:
mat=matrizes['cesnet']
print("matriz:\n")
print(mat)

matriz:

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]


<h1>Dijkstra</h1>
<p>Implementação da busca pelo menor caminho</p>

In [21]:
from math import inf

letra = lambda x:chr(ord('a')+x)

class BuscaMatriz():
    #matrizNova=list()
    def __init__(self,matrix):
        self.matrix=matrix
        self.custo=[inf]*matrix.size()
        self.pai=[-1]*matrix.size()
    def backtrack(self,destino):
        cam=list()
#         print(destino)
        pai=self.pai[destino]
        cam.append(destino)
        if(pai!=-1):
            cam = cam + self.backtrack(pai)
        return cam

class GraphMatrix():
    def __init__(self,first,matrix):#de certa forma isto não deveria estar aqui e desta forma
        self.actual=first
        self.matrix=matrix
        self.custo=0
        self.visited=[0]*len(matrix)
        self.visited[first]=1
#         for i,linha in enumerate(matrix):
#             print ("linha %d: %s"%(i,linha))
    def __str__(self):
        return 'teste'
    def visit(self,aresta,custo):
        if (self.visited[aresta]==1):
            return False
        self.visited[aresta]=1
        self.actual=aresta
        self.custo+=custo
        return True
    def get_actual(self):
        return self.actual
    def set_actual(self,actual):
        self.actual=actual
    def size(self):
        return len(self.matrix)
    def run(self,*reverse): #nem isto
#         print("Atualmente em %d com custo %d" % (self.actual,self.custo))
        vizinhos=self.get_vizinhos(self.actual)
#         print ("vizinhos:")
#         for vizinho in vizinhos:
#             print("numero %d custo %d" % vizinho)
        while len(vizinhos)>0:
            if reverse:
                v=vizinhos.pop()
            else:
                v=vizinhos.pop(0)
            if self.visit(*v) == True:
                return True
        return False

class AdjMatrix(GraphMatrix):
    def __init__(self,*args):
        super(AdjMatrix,self).__init__(*args)
    def get_vizinhos(self,vert):
#         print(self.matrix)
        vizinhos=[(i,ar) for i,ar in enumerate(self.matrix[vert]) if ar>0 ]
        vizinhos.sort(key=lambda tupla:tupla[1])
        return vizinhos

In [22]:
#import mtx /\
import sys
from math import inf
class Dijkstra(BuscaMatriz):
    show=lambda self:print("Custo: %s\nPai: %s\nFila %s\nAtual: %d" % (self.custo,self.pai,self.fila,self.matrix.get_actual()))
    def __init__(self,matrix):
        super(Dijkstra,self).__init__(matrix)
        self.fila=list(range(0,matrix.size()))
        self.fila.remove(self.matrix.get_actual())

    def explora(self):
        index=self.matrix.get_actual()
        if(self.custo[index]==inf):#caso seja desconexo
            self.custo[index]=0
        for vizinho in self.matrix.get_vizinhos(index):
            (vert,value)=vizinho
#             self.show()
            nvalue=value+self.custo[index]
            if nvalue < self.custo[vert]:
#                 print("Novo valor para %d: %d"%(vert,nvalue))
                self.atualiza(vert,nvalue)
    def gen_fila(self):
        for x in self.fila:
            custo=self.custo[x]
            yield (custo,x)
    def end(self):
        return self.fila==[]
    def atualiza(self,filho,valor):
        self.custo[filho]=valor
        self.pai[filho]=self.matrix.get_actual()
    def choose(self):
        fila=list()
        for a in self.gen_fila():
            fila.append(a)
        fila.sort()
#         print("\nFila em ordem de peso: %s"%(fila))
        (peso,actual)=fila.pop(0)
        
        self.matrix.set_actual(actual)
        self.fila.remove(actual)

<h2>Calcular as rotas</h2>
<p>Ao explorar a matriz verificamos os menores custos para comunicação entre todos os nós.</p>

Suporte a threads para acelerar o processo:

In [23]:
from threading import Lock #bloqueios mutex
import concurrent.futures #threads

mutex = Lock()

Explorar todos caminhos da árvore

In [24]:
# para cada caminho uma thread, a fim de acelerar o processo de exploracao
def explorar_caminho(ijcl):
    i,j,caminhos,lmtx=ijcl
    di=Dijkstra(AdjMatrix(i,lmtx))
    di.explora()
    while(di.end()==False):
        di.explora()
        di.choose()
    mutex.acquire()
    try:
        caminhos.update({(j,i):di.backtrack(j)})
    finally:
        mutex.release()
    print(caminhos)
    return caminhos

def explorar_matrix(mtx):
    caminhos = dict()
    #lmtx = mtx.tolist()
    lmtx = mtx
    # gera lista de entradas para as threads com todas as combinacoes de origem destino:
    t_caminhos=[(i,j,caminhos,lmtx) for i in range(len(mtx)) for j in range(len(mtx)) if i!=j]
    with concurrent.futures.ThreadPoolExecutor() as executor: # chamada paralelizada dos processos:
        executor.map(explorar_caminho,t_caminhos)
    return caminhos


In [25]:
matrix = np.matrix(matrizes.get('portugal'))#nova instancia
caminhos = explorar_matrix(matrix.tolist())

{(1, 0): [1, 7, 19, 14, 21, 0]}
{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0]}{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0]}

{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0]}
{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0]}
{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0]}{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0], (7, 0): [7, 19, 14, 21, 0]}
{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0], (7, 0): [7

{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0], (7, 0): [7, 19, 14, 21, 0], (8, 0): [8, 9, 0], (9, 0): [9, 0], (10, 0): [10, 2, 9, 0], (11, 0): [11, 14, 21, 0], (12, 0): [12, 6, 16, 8, 9, 0], (13, 0): [13, 5, 14, 21, 0], (14, 0): [14, 21, 0], (15, 0): [15, 14, 21, 0], (16, 0): [16, 8, 9, 0], (17, 0): [17, 22, 0], (18, 0): [18, 1, 7, 19, 14, 21, 0], (19, 0): [19, 14, 21, 0], (20, 0): [20, 7, 19, 14, 21, 0], (21, 0): [21, 0], (22, 0): [22, 0], (23, 0): [23, 3, 20, 7, 19, 14, 21, 0], (24, 0): [24, 25, 7, 19, 14, 21, 0], (25, 0): [25, 7, 19, 14, 21, 0], (0, 1): [0, 21, 14, 5, 13, 1], (2, 1): [2, 9, 8, 16, 19, 7, 1], (3, 1): [3, 18, 1], (4, 1): [4, 3, 18, 1], (5, 1): [5, 13, 1], (6, 1): [6, 12, 25, 7, 1], (7, 1): [7, 1], (8, 1): [8, 16, 19, 7, 1], (9, 1): [9, 8, 16, 19, 7, 1], (10, 1): [10, 17, 11, 14, 5, 13, 1], (11, 1): [11, 14, 5, 13, 1], (12, 1): [12, 25, 7, 1], 

{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0], (7, 0): [7, 19, 14, 21, 0], (8, 0): [8, 9, 0], (9, 0): [9, 0], (10, 0): [10, 2, 9, 0], (11, 0): [11, 14, 21, 0], (12, 0): [12, 6, 16, 8, 9, 0], (13, 0): [13, 5, 14, 21, 0], (14, 0): [14, 21, 0], (15, 0): [15, 14, 21, 0], (16, 0): [16, 8, 9, 0], (17, 0): [17, 22, 0], (18, 0): [18, 1, 7, 19, 14, 21, 0], (19, 0): [19, 14, 21, 0], (20, 0): [20, 7, 19, 14, 21, 0], (21, 0): [21, 0], (22, 0): [22, 0], (23, 0): [23, 3, 20, 7, 19, 14, 21, 0], (24, 0): [24, 25, 7, 19, 14, 21, 0], (25, 0): [25, 7, 19, 14, 21, 0], (0, 1): [0, 21, 14, 5, 13, 1], (2, 1): [2, 9, 8, 16, 19, 7, 1], (3, 1): [3, 18, 1], (4, 1): [4, 3, 18, 1], (5, 1): [5, 13, 1], (6, 1): [6, 12, 25, 7, 1], (7, 1): [7, 1], (8, 1): [8, 16, 19, 7, 1], (9, 1): [9, 8, 16, 19, 7, 1], (10, 1): [10, 17, 11, 14, 5, 13, 1], (11, 1): [11, 14, 5, 13, 1], (12, 1): [12, 25, 7, 1], 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
print(matrix)

[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 1 

<p>O indice de wiener é expresso pelo comprimento de todos os caminhos curtos descobertos.</p>

In [27]:
def wiener(caminhos,mtx):
    custo = 0
    for i in range(len(mtx)):
        for j in range(len(mtx)):
            if(i<j):
                custo+= (len(caminhos[i,j]) -1)
    return custo

In [28]:
print(caminhos)
print("Índice Wiener: {}".format(wiener(caminhos,matrix.tolist())),file=sys.stderr)

{(1, 0): [1, 7, 19, 14, 21, 0], (2, 0): [2, 9, 0], (3, 0): [3, 20, 7, 19, 14, 21, 0], (4, 0): [4, 3, 20, 7, 19, 14, 21, 0], (5, 0): [5, 14, 21, 0], (6, 0): [6, 16, 8, 9, 0], (7, 0): [7, 19, 14, 21, 0], (8, 0): [8, 9, 0], (9, 0): [9, 0], (10, 0): [10, 2, 9, 0], (11, 0): [11, 14, 21, 0], (12, 0): [12, 6, 16, 8, 9, 0], (13, 0): [13, 5, 14, 21, 0], (14, 0): [14, 21, 0], (15, 0): [15, 14, 21, 0], (16, 0): [16, 8, 9, 0], (17, 0): [17, 22, 0], (18, 0): [18, 1, 7, 19, 14, 21, 0], (19, 0): [19, 14, 21, 0], (20, 0): [20, 7, 19, 14, 21, 0], (21, 0): [21, 0], (22, 0): [22, 0], (23, 0): [23, 3, 20, 7, 19, 14, 21, 0], (24, 0): [24, 25, 7, 19, 14, 21, 0], (25, 0): [25, 7, 19, 14, 21, 0], (0, 1): [0, 21, 14, 5, 13, 1], (2, 1): [2, 9, 8, 16, 19, 7, 1], (3, 1): [3, 18, 1], (4, 1): [4, 3, 18, 1], (5, 1): [5, 13, 1], (6, 1): [6, 12, 25, 7, 1], (7, 1): [7, 1], (8, 1): [8, 16, 19, 7, 1], (9, 1): [9, 8, 16, 19, 7, 1], (10, 1): [10, 17, 11, 14, 5, 13, 1], (11, 1): [11, 14, 5, 13, 1], (12, 1): [12, 25, 7, 1], 

Índice Wiener: 1183


In [29]:
rotear=lambda caminhos,origem,destino: [par for par in zip(caminhos[(origem,destino)],caminhos[(origem,destino)][1:]) ]

In [30]:
print(f"grau: {calc_grau(matrix)}")
print("caminho:\n{}".format(caminhos[(2,5)]))
print("enlaces:\n{}".format(rotear(caminhos,2,5)))

grau: 2.769230769230769
caminho:
[2, 9, 0, 21, 14, 5]
enlaces:
[(2, 9), (9, 0), (0, 21), (21, 14), (14, 5)]


In [31]:
aumentar_grau(matrix,4)
print (matrix)
print (calc_grau(matrix))

[[0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1]
 [1 0 1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 1 0 1 

In [32]:
%%capture --no-stderr
# print(f'calculando indice wiener:\n {caminhos}')
caminhos = explorar_matrix(matrix.tolist())

print(f'Wiener: {wiener(caminhos,matrix)}',file=sys.stderr)

Wiener: 608


In [33]:
print("caminhos:\n{}".format(caminhos[(2,5)]))
print("enlaces:\n{}".format([par for par in zip(caminhos[(2,5)],caminhos[(2,5)][1:]) ]))

caminhos:
[2, 9, 5]
enlaces:
[(2, 9), (9, 5)]


<h1>Executar os testes e simulação</h1>
<p>Por fim, para simular a utilização da rede executa-se diversas iterações contando o número bloqueios em cada iteração, a cada iteração o grau médio é aumentado. Por fim, calcula-se a correlação entre a taxa de bloqueio e o grau médio.</p>
<p>Segue abaixo nossa representação da classe que instância cada simulação:</p>

In [34]:
class sim_rede():
    def __init__(self,nome,matrix_np,ncanais=-1):
        self.nome = nome
#         self.matrix = np.matrix(matrix_np) #criar uma nova cópia das originais
        if(ncanais<1):
            ncanais = int(len(matrix_np))
        self.ncanais = ncanais
        self.canais = criar_canais(matrix_np,ncanais)
        
        self.matrix = matrix_np.tolist() # nova instância do tipo lista
        self.caminhos = explorar_matrix(self.matrix) #aqui monta a árvore usando a matrix tipo lista
        self.bloqueios = 0
        self.sucesso = 0
        self.sum_demandas = 0

    def testa_canais(self,enlaces):# testa os 100% ocupados para exclusão da matriz
        return sum(self.canais[no]) # Se todos os canais
    
    ########################## Conexão #################################
    # calcula os enlaces que ligam a origem ao destino
    def calc_enlaces(self,origem,destino):
        caminho = self.caminhos[(origem,destino)]
        rota = [par for par in zip(caminho,caminho[1:]) ]
        print("canais:{}, origem:{}, destino:{}, rota:{}, caminho: {}".format(self.ncanais,origem,destino,rota,caminho))
        return rota    
    
    # Conecta origem e destino:
    
    def conectar(self,origem,destino):
        ok=1
        enlaces = self.calc_enlaces(origem,destino)

        caminho = self.caminhos[(origem,destino)]
        
        # excluir adjacencias relativas aos canais interrompidos e recalcular os caminhos >Sob Demanda<
        # ou seja, somente os caminhos que usam estes canais serão recalculados cada vez que necessário
#         excluir = []
        for o,d in enlaces:
            if sum(self.canais[o,d]) == 0: #se igual a 0 todos os canais do enlace estão ocupados
#                 excluir+=[(o,d),(d,o)]
                self.matrix[o][d] = 0 #exclui enlaces com canais cheios da matriz
                self.matrix[d][o] = 0
        # Atualiza caminhos ida e volta
        explorar_caminho((origem,destino,self.caminhos,self.matrix))
        explorar_caminho((destino,origem,self.caminhos,self.matrix))
        
        enlaces = self.calc_enlaces(origem,destino)
        for canal in range(self.ncanais): #canal 0..n
            # Atribuição de espectro: FIRST FIT
            for o,d in enlaces: #nó do caminho
#                 print(self.canais)
                ok = ok*self.canais[(o,d)][canal] # se zero == bloqueio
#             self.sum_demandas+=1
            if ok == 1:
                for no in caminho: #no do caminho
                    try:
                        self.canais[(d,o)][canal] = 0 # marca canal do enlace como ocupado
                        self.sum_demandas+=1 #caso possível a conexão aumenta a demanda
                    except: #caminho inexistente
                        print("bloqueio ({}, {})".format(origem,destino))
                        self.bloqueios += 1
                        return (False,caminho)
# Caso montássemos uma matriz de demanda, mas como só precisamos da soma dos valores dos enlaces, utilizamos
# uma variável(sum_demandas), que já calcula a dimensão da matriz de demanda.
#                     if i > j:
#                         self.demandas[(j,i)]+=1
#                     else:
#                         self.demandas[(i,j)]+=1    
                self.sucesso += 1
                return (True,caminho)
            else:
                ok=1
        print("bloqueio ({}, {})".format(origem,destino))
        self.bloqueios += 1
        return (False,caminho)
    def wiener(self):
        return wiener(self.caminhos,self.matrix)

<p>Exemplo de uma rede pequena com dois canais com seu grau médio original provocava entre 3 e 4 bloqueio. vejamos o resultado ao incrementar duas vezes o grau.</p>

In [35]:
n='scteste'
matrix = np.matrix(matrizes.get(n)) #cópia instanciada
for i in range(3):
    print("\n===========iteracao ({})============\n".format(i))
    rede = sim_rede(n,matrix,2)
    print("wiener antes: {}".format(rede.wiener()))
    # neste teste conectamos todas as entradas com todas as saídas:
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if(i<j):
    #             print(rede.caminhos[i,j])
                rede.conectar(i,j)
    print("demandas:{}, rede:{}, bloqueios:{}, wiener: {}, grau: {}".format(rede.sum_demandas,n,rede.bloqueios,rede.wiener(),calc_grau(matrix)))
    aumentar_grau(matrix,1)
#     print("demandas(total={}):\n{}".format(rede.sum_demandas,rede.demandas))
    print("matriz:\n{}".format(matrix))


===========iteracao (0)============

{(1, 0): [1, 0]}
{(1, 0): [1, 0], (2, 0): [2, 0]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0], (0, 1): [0, 1]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0], (0, 1): [0, 1], (2, 1): [2, 0, 1]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0], (0, 1): [0, 1], (2, 1): [2, 0, 1], (3, 1): [3, 1]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0], (0, 1): [0, 1], (2, 1): [2, 0, 1], (3, 1): [3, 1], (4, 1): [4, 3, 1]}
{(1, 0): [1, 0], (2, 0): [2, 0], (3, 0): [3, 1, 0], (4, 0): [4, 3, 1, 0], (5, 0): [5, 2, 0], (0, 1): [0, 1], (2, 1): [2, 0, 1], (3, 1): [3, 1], (4, 1

<h1>Simulação e cálculo de correlação</h1>


<p>Por fim simula-se o uso da rede:</p>

In [36]:
# instanciação das listas para coletar os dados
ind_wiener = dict()
bloqs = dict()
grau = dict()
demandas = dict()
copias = dict()
tentativas = dict()
sucesso = dict()
n_wiener = dict()
n_bloqs = dict()
n_demand = dict()
correlacao=dict()
inc_grau = 1 #incremento do grau

instanciar listas de relatórios:

In [37]:
def init_listas(n):
    ind_wiener.update({n:[]})
    bloqs.update({n:[]})
    grau.update({n:[]})
    copias.update({n:np.matrix(matrizes.get(n))}) #copias das matrizes originais
    demandas.update({n:[]})
    tentativas.update({n:[]})
    sucesso.update({n:[]})
    n_wiener.update({n:[]})
    n_bloqs.update({n:[]})
    n_demand.update({n:[]})

In [53]:
mutex2 = Lock()
class Counter:
    def __init__ (self):
        self.count=0
        self.redes=[]
    def up(self,rede):
        self.redes.append(rede)
        self.count+=1
    def down(self,rede):
        self.redes.remove(rede)
        self.count-=1
    def list_redes(self):
        return ', '.join(self.redes)
counter = Counter()
def simulacao (n):
    #inicia listas vazias para o relatório
    init_listas(n)
    #contador interno de iterações
    it=0
    g=0
    
    mutex2.acquire()
    try: #evitar prints bagunçados e contar instâncias
        counter.up(n)
        print(f"redes:{counter.list_redes()} [{counter.count}]",file=sys.stderr)
    finally:
        mutex2.release()
        
    while g < 5:
        g = calc_grau(copias[n])
        rede = sim_rede(n,copias[n])
#         print (rede.ncanais,file=sys.stderr)
#         print(copias[n],file=sys.stderr)
#         print(f'iteração: {it}',file=sys.stderr)
        ind_wiener[n] += [rede.wiener()]
        grau[n] += [calc_grau(copias[n])]
        conexoes = int((len(rede.matrix))**2) # metade numero de interconexões é o numero de nós na portencia de 2
#         conexoes = len(rede.matrix)*10
        for l in range(conexoes): 
            j = randint(0,len(matrix)-1)
            i = randint(0,len(matrix)-1)
            while j == i: #origem e destino devem ser diferentes
                j = randint(0,len(matrix)-1)
            rede.conectar(i,j)
        tentativas[n] += [conexoes]
        bloqs[n] += [rede.bloqueios]
        sucesso[n] += [rede.sucesso]
        demandas[n] += [rede.sum_demandas]
#         print("rede:{}, grau:{}, bloqueios:{}, wiener: {},conexões: {}, sucesso: {}".format(n,grau[n][it],rede.bloqueios,ind_wiener[n][it],conexoes,rede.sucesso),file=sys.stderr)
        if calc_grau(copias[n]) == (len(copias[n])-1): #grau máximo para
            break
        aumentar_grau(copias[n],inc_grau)
        it += 1
        
    mutex2.acquire()
    try: #evitar prints bagunçados
        counter.down(n)
        print(f"redes:{counter.list_redes()} [{counter.count}]",file=sys.stderr)
    finally:
        mutex2.release()

Simular uma rede isolada:

In [54]:
%%capture --no-stderr
teste = 'scteste'
# teste = 'OmnicomEurope'
# teste = 'usaGde'
# teste = 'cesnet'
n = teste
simulacao (teste)

redes:scteste [1]
redes: [0]


In [55]:
# Normalização das entradas wiener e blocks para adequar todas as redes(não utilizado, trocas de sinais atrapalham o calculo de correlação da TxB com o índice de Wiener):
# # def percentual (vin):
#     minimo = np.min(vin)
#     maximo = np.max(vin)
#     return  ( (vin - np.min(vin)) * 100) / (maximo - minimo)
# def normalizar (lista):
#     n_lista = []
#     mean = np.mean(lista)
#     std = np.std(lista)
#     for v in lista:
#         n_lista.append((v - mean)/std)
#     n_lista = percentual(n_lista)
# #         n_lista.append((v - mean)/std)

#     return n_lista
# def norm_rede(n):
#     n_wiener.update({n:normalizar(ind_wiener[n]) })
#     n_bloqs.update({n:normalizar(bloqs[n])})
#     n_demand.update({n:normalizar(demandas[n])})


Formatação do print de saída

In [56]:
def print_rede(n):
#     norm_rede(n)
    tb = []
    print(f'{"wiener":>10}{"demand":>10}{"Blocks":>10}{"grau":>10}{"tentativas":>12}{"sucesso":>10}{"tx de b":>10}') #{"nd":>10}{"nw":>10}{"nb":>10}')
    for w,d,b,g,t,s in zip(ind_wiener[n],demandas[n],bloqs[n],grau[n],tentativas[n],sucesso[n]): #,nw,nb,nd in.. #,n_wiener[n],n_bloqs[n],n_demand[n]):
        print (f"{w:10}{d:10}{b:10}{g:10.2f}{t:12}{s:10}{b/d:10.7f}") #{nd:>10.2f}{nw:>10.2f}{nb:>10.2f}")
        tb.append(b/d)
#         tb.append(nb/nd)
    print(f"\ncorrelação de TX de bloqueio/wiener")
    correlacao[n]=corr(tb,ind_wiener[n],True)
#     corr(tb,n_wiener[n],True)

In [57]:
print_rede(n)

    wiener    demand    Blocks      grau  tentativas   sucesso   tx de b
        25        77         6      2.33          36        30 0.0779221
        20        85         0      3.33          36        36 0.0000000
        16        72         0      4.67          36        36 0.0000000
        15        72         0      5.00          36        36 0.0000000

correlação de TX de bloqueio/wiener
Taxa de correlação entre [x] e [y] é:
0.46753246753246747
------------------- =  0.8798826901281196
0.5313577284539944


In [58]:
%%capture --no-stderr
# /\ suprime a saída pode atribuir a uma var: %capture [--no-stderr] [--no-stdout] [--no-display] [output]

# Preparar as listas para receberem os resultados

print("Precione S para executar, T para executar tbm usaGde:",file=sys.stderr)
resposta = input().lower()
if resposta == 's' or resposta =='t':
    #inicia listas vazias para os relatorios
    init_listas(n)

    if resposta == 't':
        todos = names
    else:
        todos = [n for n in names if n!='usaGde']
        
    #simula cada rede em uma thread diferente
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(simulacao,todos)
else:
    print("Cancelado",file=sys.stderr)
# for n in [n for n in names if n!='usaGde']: #paralelizei internamente cada rede
#     simulacao(n)

Precione S para executar, T para executar tbm usaGde:


s


redes:OmnicomEurope [1]
redes:OmnicomEurope, arpanet [2]
redes:OmnicomEurope, arpanet, bren [3]
redes:OmnicomEurope, arpanet, bren, austria [4]
redes:OmnicomEurope, arpanet, bren, austria, canarie [5]
redes:OmnicomEurope, arpanet, bren, austria, canarie, arnes [6]
redes:OmnicomEurope, arpanet, bren, austria, canarie, arnes, cesnet [7]
redes:OmnicomEurope, arpanet, bren, austria, canarie, arnes, cesnet, coxUsa [8]
redes:OmnicomEurope, arpanet, bren, austria, canarie, arnes, cesnet, coxUsa, deutschTelecom [9]
redes:OmnicomEurope, arpanet, bren, austria, canarie, arnes, cesnet, coxUsa, deutschTelecom, eon [10]
redes:OmnicomEurope, arpanet, austria, canarie, arnes, cesnet, coxUsa, deutschTelecom, eon [9]
redes:OmnicomEurope, arpanet, austria, canarie, arnes, cesnet, coxUsa, deutschTelecom, eon, geant2 [10]
redes:OmnicomEurope, arpanet, austria, canarie, arnes, coxUsa, deutschTelecom, eon, geant2 [9]
redes:OmnicomEurope, arpanet, austria, canarie, arnes, coxUsa, deutschTelecom, eon, geant2,

In [59]:
names.sort()
correlacao=dict()
for n in todos:
    print (f"\n{n}({len(matrizes[n])}):")
    print_rede(n)


OmnicomEurope(38):
    wiener    demand    Blocks      grau  tentativas   sucesso   tx de b
      3166      1367       987      2.84        1444       457 0.7220190
      1938      1743       897      3.84        1444       547 0.5146299
      1693      1656       860      4.84        1444       584 0.5193237
      1530      1629       861      5.84        1444       583 0.5285451

correlação de TX de bloqueio/wiener
Taxa de correlação entre [x] e [y] é:
215.35921999576476
------------------ =  0.9601478656225509
224.29797295453847

arnes(17):
    wiener    demand    Blocks      grau  tentativas   sucesso   tx de b
       411       366       179      2.35         289       110 0.4890710
       300       551       123      3.41         289       166 0.2232305
       263       542       111      4.47         289       178 0.2047970
       236       575        95      5.53         289       194 0.1652174

correlação de TX de bloqueio/wiener
Taxa de correlação entre [x] e [y] é:
33.429691

<h2 style='font-family: "Comic Sans MS", "Comic Sans", cursive;font-size: 30px;font-style: italic' >Correlação Total:</h2>
Uma forma de generalizar a correlação de todas as redes foi ponderando conforme o tamanho. Visto que a expanção da rede tem custos relativos a sua abrangência achamos que possa ser uma abordagem válida.

In [61]:
ptotal = sum([len(matrizes.get(n)) for n in todos])
print( f'Chegamos neste valor: {sum ([ (correlacao[n] * len(matrizes.get(n)))/ptotal for n in todos])*100:.2f}')

Chegamos neste valor: 85.36


No geral há uma boa diferença de desempenho ao adicionar novos enlaces(aumentar o grau). Porém quando a rede está com disponibilidade suficiente pode não se fazer necessário(taxa de bloqueio 0). Outra observação a ser feita é que quantidade de conexões com sucesso de fato se concretiza se comparar com a Taxa de Bloqueio.